<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GPT_finetunning_aviation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/docs/guides/fine-tuning

In [ ]:
!pip install colab-env --upgrade -q
!pip install openai -q

In [2]:
import colab_env
import os

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))


Mounted at /content/gdrive


In [3]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'),
 Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='text-embedding-3-small', created=1705948997

In [4]:
#modellist.data[33] # Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system'),
#modellist.data[34]  # Model(id='ada:ft-personal-2023-07-11-12-33-39', created=1689078819, object='model', owned_by='user-mgtzrbiiso3g68oicniq0obp'),
len(modellist.data) #34 Official by OpenAI

34

Fine-tuning is currently available for the following models: gpt-3.5-turbo-0125 (recommended), gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, babbage-002, davinci-002, and gpt-4-0613 (experimental).

In [5]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {
      "role": "system",
      "content": "Summarize content you are provided with for a second-grade student."
    },
    {
      "role": "user",
      "content": "Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus."
    }
  ],
  temperature=0.7,
  max_tokens=64,
  top_p=1
)

In [6]:
print(response.choices[0].message.content)

Jupiter is a big planet in our Solar System, it is the fifth one from the Sun. It is really huge and made of gas. It is so big that it has more mass than all the other planets combined. People have known about Jupiter for a very long time, even before they wrote things down. Jupiter


In [ ]:
#get SQuAD v2
#!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
#!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

https://github.com/frank-morales2020/Cloud_curious/blob/master/mistral_finetunning_squad2_0.ipynb

AVIATION-QA

In [7]:
import json

def get_data(path):
    #read each file and retrieve the contexts, qustions and answers
    data = []
    with open(path, 'r') as f:  # Open in text mode for JSON Lines
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding line: {line}")
                print(e)
    return data  # Return the list of parsed JSON objects

In [25]:
# Upload the training file FOR SQuAD v2 1000 QUESTIONS
#filename_path='/content/gdrive/MyDrive/datasets/dataft-L1000.jsonl'
# aviation 10 000
filename_path='/content/gdrive/MyDrive/datasets/aviation.jsonl'
data=get_data(filename_path)
#data[10]

In [26]:
numberofquestions=len(data)
numberofquestions

100

In [40]:
nrec=4
data[nrec]['messages'][0]['content']

question=data[nrec]['messages'][1]['content']
answer=data[nrec]['messages'][2]['content']

print('Record #: %s'%nrec)
print('Question: %s'%question)
print('Answer by the dataset: %s'%answer)

Record #: 4
Question: In the accident with no. DCA16CA009, is the aircraft damaged?
Answer by the dataset: Not possible to get or use


In [28]:
from openai import OpenAI
client = OpenAI()

# Upload the training file
file_response = client.files.create(
  file=open(filename_path, "rb"),
  purpose="fine-tune"
)

# Get the file ID
file_id = file_response.id

# Start the fine-tuning job using the file ID
cftjc=client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo-0125"
)

training_file=cftjc.training_file
fine_tuning_job=cftjc.id

In [ ]:
# List 10 fine-tuning jobs
#client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
#client.fine_tuning.jobs.retrieve(fine_tuning_job)

# Cancel a job
#client.fine_tuning.jobs.cancel("ftjob-HHsRv4STcKEaAywwrYDjL4zI")

# List up to 10 events from a fine-tuning job
#client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tuning_job, limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
#client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

In [29]:
actual_job_id = fine_tuning_job  # Replace with a real ID
fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
client.fine_tuning.jobs.retrieve(actual_job_id)

FineTuningJob(id='ftjob-SQiodyr12za98rKCwRTyS6hG', created_at=1717726451, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-kw9OnWtmHIJ748Xe7oTALKxl', result_files=[], seed=810225732, status='running', trained_tokens=None, training_file='file-mz67MuVcOUJSxJgIOlC5LfeC', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [30]:
organization_id=fine_tuning_job_retrieve.organization_id
fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model
print(fine_tuned_model)

None


https://platform.openai.com/finetune/

In [31]:
while fine_tuned_model == None:
       fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
       organization_id=fine_tuning_job_retrieve.organization_id
       fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model


In [ ]:
print('Fine Tuned Model: %s'%fine_tuned_model)

Fine Tuned Model: ft:gpt-3.5-turbo-0125:personal::9WaztzN2


In [36]:
from openai import OpenAI
client = OpenAI()


completion = client.chat.completions.create(
  model=fine_tuned_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": question}
  ]
)
print('PROMPT: %s'%question)
print('ANSWER GENERATED: %s'%completion.choices[0].message.content)

PROMPT: In the accident with no. DCA16CA009, is the aircraft damaged?
ANSWER GENERATED: Yes, in the accident with no. DCA16CA009, the aircraft is damaged.


In [37]:
modellist=client.models.list()
modellist.data

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'),
 Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='text-embedding-3-small', created=1705948997

In [38]:
modellist.data[33]

Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system')

In [39]:
for i in range(34,len(modellist.data)):
  print(modellist.data[i].id)
  client.models.delete(modellist.data[i].id)

ft:gpt-3.5-turbo-0125:personal::9XJZ2ZUj:ckpt-step-100
ft:gpt-3.5-turbo-0125:personal::9XJZ2GW8:ckpt-step-200
ft:gpt-3.5-turbo-0125:personal::9XJZ208T
